In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import torch

In [9]:
def create_balanced_grids(df_res, num_grid,balance:list = ['topic', 'fake']):
  '''Create grids balanced by topic and fake/true
  Input: num_girds, pd.DataFrame with in each row an image. Each column is a feature. Mandatory columns are topic and the labelcolumn
  Output: return a dictionary where each key is the index of the grid and the value is a list with the name of the images '''
  
  dict_grid = dict()
  df_copy = df_res.copy()
  groups = df_copy.groupby(balance)
  classes={}
  classes_count={}
  #crea per ogni gruppo le classi in base a cred e manip per potere usare tutte quelle meno frequenti
  for group in groups: 
    #temp=group[1].groupby(['cred','manip'])
    temp=group[1].groupby(['authenticity'])
    classes[group[0]]={}
    classes_count[group[0]]={}
    for dclass in temp:
      #classes[group[0]][dclass[0]]=[(dclass[1]["folder"][i]+os.path.sep +i) for i in dclass[1].index.tolist()]      
      classes[group[0]][dclass[0]]=[(dclass[1]['path_drive_not_annotated'][i]+os.path.sep +i) for i in dclass[1].index.tolist()]
      classes_count[group[0]][dclass[0]]=[0,len(classes[group[0]][dclass[0]])]
      
  
  listofimages=[]
  for grid in range(num_grid):
    # print("grid:",grid)  
    dict_grid[grid] = []
    for group in groups:
      min_el=1000
      min_classes=[]
      for dclass_k, dclass_v in classes[group[0]].items():
        # print(group[0],'len',len(dclass[1]))
        lendclass=classes_count[group[0]][dclass_k][0]
        if lendclass<min_el and lendclass < classes_count[group[0]][dclass_k][1]:
          #print("lendclass",lendclass,"classes_count",classes_count[group[0]][dclass_k][1])
          min_el=lendclass
          min_classes=[dclass_k]
          classes_counts=classes_count[group[0]][dclass_k][1]
        elif lendclass==min_el and lendclass < classes_count[group[0]][dclass_k][1]:
          min_classes.append(dclass_k)
      sel_class_k= random.choice(min_classes)
      sel_class_v=classes[group[0]][sel_class_k]
      # print("sellendclass",classes_count[group[0]][sel_class_k][0],"sel_classes_count",classes_count[group[0]][sel_class_k][1])    
      # print("selected class pre",sel_class_v)
      topic_sample =random.choice(sel_class_v)
      # print("topic_sample",topic_sample)
      # print(group[0],"sel class pre",sel_class_k,len(sel_class_v))
      sel_class_v.remove(topic_sample)
      #print(group[0],"sel class post",sel_class_k,len(sel_class_v))
      classes_count[group[0]][sel_class_k][0]+=1
      dict_grid[grid].append(topic_sample)
      listofimages.append(topic_sample)
    #print("grid",dict_grid[grid])
    random.shuffle(dict_grid[grid])
    #print("grid",dict_grid[grid])

  #assert (len(list(pd.DataFrame.from_dict(dict_grid).values.reshape(-1)))  == len(set(list(pd.DataFrame.from_dict(dict_grid).values.reshape(-1))))), 'Huston Problem'
  # print( len(listofimages))
  # print(listofimages)
  # print( len(set(listofimages)))
  # print(set(listofimages))
  # print("duplicates", [k for k,v in Counter(listofimages).items() if v>1])
  assert(len(listofimages)==len(set(listofimages)))
  random.shuffle(dict_grid)
  return dict_grid

In [14]:
path_fra = '/content/drive/MyDrive/Qualtrics_Annotator/DEF_images_data.xlsx'
df = pd.read_excel(path_fra, index_col=0)
df.head(4)

,topic,t_f,authenticity,url,virality,interpret,NOT_ann_qualtrics_url,path_drive_not_annotated,id_qualtrics_not_annotated_img,annotated_name,annotated_qualtrics_name,start_id,middle_id,end_id,annotated_qualttrics link,ita_trad,EST
space_rover_sistem_TRUE.jpg,space,T,T,https://www.jpl.nasa.gov/news/twin-of-nasas-pe...,4.027092,"Updated with new features, including additiona...","<img src=""https://qfreeaccountssjc1.az1.qualtr...",/content/drive/MyDrive/Resized_images/space/sp...,IM_9B6a36NQaVHeL78,annoted_space_rover_sistem_TRUE.jpg,IM_3qLneJI5SNDf6tM,"<img src=""",https://qfreeaccountssjc1.az1.qualtrics.com/Co...,"""/>","<img src=""https://qfreeaccountssjc1.az1.qualtr...","Aggiornato con nuove funzionalità, tra cui sof...",INTERPRET
nasa_fake_mars_rover.jpg,space,F,T,https://edition.cnn.com/2016/07/15/health/mars...,3.040368,NASA’s Mars 2020 rover will explore the Red Pl...,"<img src=""https://qfreeaccountssjc1.az1.qualtr...",/content/drive/MyDrive/Resized_images/space/na...,IM_eMcmqjB7SmJAPhY,annoted_nasa_fake_mars_rover.jpg,IM_06wFfIRn2n8NJXg,"<img src=""",https://qfreeaccountssjc1.az1.qualtrics.com/Co...,"""/>","<img src=""https://qfreeaccountssjc1.az1.qualtr...",Il rover Mars 2020 della NASA esplorerà il Pia...,INTERPRET
true_hubble_space_nasa.jpg,space,T,T,https://web.astropiombino.org/web/wp-content/u...,1.949575,Hubble Looks at Helix Nebula,"<img src=""https://qfreeaccountssjc1.az1.qualtr...",/content/drive/MyDrive/Resized_images/space/tr...,IM_3UBq6B8oHKLJ6Ps,annoted_true_hubble_space_nasa.jpg,IM_d0Ymfvu1zYnRvWm,"<img src=""",https://qfreeaccountssjc1.az1.qualtrics.com/Co...,"""/>","<img src=""https://qfreeaccountssjc1.az1.qualtr...",Hubble osserva la nebulosa dell'elica,NO_INTERPRET
ppDaE4bmoY98e2J7LezDvK.jpg,space,T,T,https://www.nbcnews.com/mach/science/here-s-wh...,2.927303,"Elon Musk posted this ""last pic"" of the red Te...","<img src=""https://qfreeaccountssjc1.az1.qualtr...",/content/drive/MyDrive/Resized_images/space/pp...,IM_37YqrzryHAoBBRk,annoted_ppDaE4bmoY98e2J7LezDvK.jpg,IM_3fRwOrmvC1on2D4,"<img src=""",https://qfreeaccountssjc1.az1.qualtrics.com/Co...,"""/>","<img src=""https://qfreeaccountssjc1.az1.qualtr...","Elon Musk ha pubblicato questa ""ultima foto"" d...",INTERPRET


In [15]:
df.groupby(['topic',	't_f']).size()

topic            t_f
commons          F      66
                 T      58
naturaldisaster  F      44
                 T      74
politics         F      47
                 T      91
space            F      49
                 T      55
dtype: int64

In [16]:
grids = create_balanced_grids(df, num_grid=35,balance=['topic', 't_f'])

In [17]:
grids

{0: ['/content/drive/MyDrive/strange_picturesstrange21.PNG/strange21.PNG',
  '/content/drive/MyDrive/Resized_images/naturaldisaster/image.jpg/image.jpg',
  '/content/drive/MyDrive/Resized_images/commons/bench.jpg/bench.jpg',
  '/content/drive/MyDrive/Resized_images/space/space-weed.jpg/space-weed.jpg',
  '/content/drive/MyDrive/Resized_images/politics/5f77363f220000ca2082beb9.jpeg/5f77363f220000ca2082beb9.jpeg',
  '/content/drive/MyDrive/Resized_images/naturaldisaster/unnatoureiffelmed.jpg/unnatoureiffelmed.jpg',
  '/content/drive/MyDrive/Resized_images/politics/baby-trump2.jpg/baby-trump2.jpg',
  '/content/drive/MyDrive/Resized_images/space/1-wise-all-sky-image-16.jpg/1-wise-all-sky-image-16.jpg'],
 1: ['/content/drive/MyDrive/Resized_images/naturaldisaster/GR-VFX-010.jpg/GR-VFX-010.jpg',
  '/content/drive/MyDrive/Resized_images/politics/120418292_334300827831704_4979709850671689467_n.jpg/120418292_334300827831704_4979709850671689467_n.jpg',
  '/content/drive/MyDrive/Resized_images/na

In [19]:
torch.save(grids, 'grids_all_imgs.pt')